In [1]:
import pandas as pd 
import ast 

In [2]:
test_address = 'AGPZnBZUxmhAtcp8XjT4n8bCia9dEYhhm16M2sfFvmTU'

In [3]:
test_wallet_timeseries = pd.read_csv('../data/processed/test_acc_timeseries_labeled.csv')

test_wallet_timeseries['timestamp'] = pd.to_datetime(test_wallet_timeseries['timestamp'])
received_tx = test_wallet_timeseries[test_wallet_timeseries['receiver']==test_address]
sent_tx = test_wallet_timeseries[test_wallet_timeseries['sender']==test_address]

In [4]:
#Total recognized transactions 

test_wallet_timeseries

total_tx = len(test_wallet_timeseries['signature'].unique())
total_tx

493

In [5]:
#Total SOL sent volume

sol_sent_vol = sent_tx['Native SOL Amount'].sum()

In [6]:
#Total SOL received volume

sol_received_vol = received_tx['Native SOL Amount'].sum()

In [7]:
# Total received volume by token

token_received_df = received_tx[received_tx['SYMBOL']!='SOL']

tokens_recieved_dict = token_received_df.groupby('SYMBOL')[['token_amount']].sum().to_dict()

In [8]:
# Total sent volume by token

token_sent_df = sent_tx[sent_tx['SYMBOL']!='SOL']

tokens_sent_dict = token_sent_df.groupby('SYMBOL')[['token_amount']].sum().to_dict()

In [9]:
# First tx time

first_tx = test_wallet_timeseries['timestamp'].min()

In [10]:
# Latest tx time

last_tx = test_wallet_timeseries['timestamp'].max()

In [11]:
test_wallet_timeseries = test_wallet_timeseries.sort_values('timestamp')

In [12]:
# Avg tx interval

avg_tx_interval = test_wallet_timeseries['timestamp'].diff().mean()
print(f'avg_tx_interval: {avg_tx_interval}')

avg_seconds = avg_tx_interval.total_seconds()
avg_minutes = avg_seconds / 60
avg_hours = avg_seconds / 3600

avg_tx_interval: 1 days 00:49:22.463414634


In [13]:
# Number of unique receivers

unique_receivers = len(test_wallet_timeseries['receiver'].unique())

In [14]:
# Number of unique senders

unique_senders = len(test_wallet_timeseries['sender'].unique())

In [15]:
wallet_analysis_dict = {
    'wallet_address':test_address,
    'entity_label': test_wallet_timeseries['wallet_entity_label'].iloc[0],
    'num_transactions': total_tx,
    'total_sol_volume_sent': sol_sent_vol,
    'total_sol_volume_received': sol_received_vol,
    'total_token_volume_sent':tokens_sent_dict,
    'total_token_volume_recieved':tokens_recieved_dict,
    'first_tx_time':first_tx,
    'last_tx_time':last_tx,
    'avg_tx_interval (seconds)':avg_seconds,
    'num_unique_senders':unique_senders,
    'num_unique_receivers':unique_receivers
}

wallet_analysis_dict

{'wallet_address': 'AGPZnBZUxmhAtcp8XjT4n8bCia9dEYhhm16M2sfFvmTU',
 'entity_label': 'Unknown Entity',
 'num_transactions': 493,
 'total_sol_volume_sent': np.float64(33.543040899999994),
 'total_sol_volume_received': np.float64(61.797667504),
 'total_token_volume_sent': {'token_amount': {'BSOL': 37.330799134,
   'PYUSD': 16103.611529806,
   'USDC': 399.830334,
   'USDT': 34.8554960850001}},
 'total_token_volume_recieved': {'token_amount': {'$WEN': 643652.0,
   'BLZE': 195.106114674,
   'BSOL': 0.46103044699999995,
   'KMNO': 95.014995437,
   'MSOL': 0.027856816,
   'PYUSD': 890.784001,
   'USDC': 3314.1448831830003,
   'USDT': 6693.240062}},
 'first_tx_time': Timestamp('2023-11-14 15:20:07'),
 'last_tx_time': Timestamp('2025-04-06 12:12:19'),
 'avg_tx_interval (seconds)': 89362.463414,
 'num_unique_senders': 52,
 'num_unique_receivers': 62}

In [16]:
wallet_analysis_df = pd.DataFrame([wallet_analysis_dict])
wallet_analysis_df

,wallet_address,entity_label,num_transactions,total_sol_volume_sent,total_sol_volume_received,total_token_volume_sent,total_token_volume_recieved,first_tx_time,last_tx_time,avg_tx_interval (seconds),num_unique_senders,num_unique_receivers
0,AGPZnBZUxmhAtcp8XjT4n8bCia9dEYhhm16M2sfFvmTU,Unknown Entity,493,33.543041,61.797668,"{'token_amount': {'BSOL': 37.330799134, 'PYUSD...","{'token_amount': {'$WEN': 643652.0, 'BLZE': 19...",2023-11-14 15:20:07,2025-04-06 12:12:19,89362.463414,52,62


# Track funding sources and flow of assets

In [17]:

# Wallet Analysis Module
class WalletAnalysis:
    def __init__(self, df):
        self.df = df
    
    def track_funding_sources_and_flow(self):
    # Track the flow of assets, both sent and received volumes (SOL and tokens)
        funding_sources = {
            "total_sol_sent": self.df['total_sol_volume_sent'].sum(),
            "total_sol_received": self.df['total_sol_volume_received'].sum(),
            "total_token_sent": self.df['total_token_volume_sent'].apply(self.extract_token_volumes).sum(),
            "total_token_received": self.df['total_token_volume_recieved'].apply(self.extract_token_volumes).sum()
        }
        return funding_sources

    
    def extract_token_volumes(self, token_dict):
        # Extract total token volumes from the dict format {'token_amount': {'token_name': amount, ...}}
        token_volumes = sum([amount for token, amount in token_dict['token_amount'].items()])
        return token_volumes
    
    def transaction_history(self):
        # Provides full transaction history per wallet
        # This is a placeholder since the actual data structure for transactions isn't fully provided
        return self.df[['wallet_address', 'num_transactions', 'first_tx_time', 'last_tx_time']]
    
    def key_activity_patterns_and_risk_factors(self):
        # Identifying key activity patterns: Average transaction interval, number of unique senders/receivers
        activity_patterns = {
            "avg_tx_interval_seconds": self.df['avg_tx_interval (seconds)'].mean(),
            "unique_senders": self.df['num_unique_senders'].sum(),
            "unique_receivers": self.df['num_unique_receivers'].sum(),
            "high_activity_risk": self._identify_risk_based_on_activity()
        }
        return activity_patterns
    
    def _identify_risk_based_on_activity(self):
        # A simple threshold logic to flag risky behavior based on high activity
        if self.df['avg_tx_interval (seconds)'].mean() < 50000:  # If transaction interval is below 14 hours
            return "High risk: Frequent transactions"
        else:
            return "Low risk: Infrequent transactions"

In [18]:
# Initialize Wallet Analysis Module
wallet_analysis = WalletAnalysis(wallet_analysis_df)


# Transaction History
transaction_history = wallet_analysis.transaction_history()
print("\nTransaction History:")
print(transaction_history)



Transaction History:
                                 wallet_address  num_transactions  \
0  AGPZnBZUxmhAtcp8XjT4n8bCia9dEYhhm16M2sfFvmTU               493   

        first_tx_time        last_tx_time  
0 2023-11-14 15:20:07 2025-04-06 12:12:19  


In [19]:
# Key Activity Patterns and Risk Factors
activity_patterns = wallet_analysis.key_activity_patterns_and_risk_factors()
print("\nActivity Patterns and Risk Factors:")
print(activity_patterns)


Activity Patterns and Risk Factors:
{'avg_tx_interval_seconds': np.float64(89362.463414), 'unique_senders': np.int64(52), 'unique_receivers': np.int64(62), 'high_activity_risk': 'Low risk: Infrequent transactions'}


In [20]:
# Track funding sources and asset flow
funding_sources = wallet_analysis.track_funding_sources_and_flow()
print("Funding Sources and Asset Flow:")
print(funding_sources)

Funding Sources and Asset Flow:
{'total_sol_sent': np.float64(33.543040899999994), 'total_sol_received': np.float64(61.797667504), 'total_token_sent': np.float64(16575.628159025), 'total_token_received': np.float64(654840.778943557)}
